In [ ]:
# 搗包
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time
import opencc

# 挑選字符

In [ ]:
字頻 = []
with open('characters_traditional_frequencies.csv', 'r',encoding='utf8') as f: # 您瞧這名字多長
    lines = f.readlines()
    print(len(lines))
    for line in lines[0:6000]:
        字頻.append(line.split(',')[0].strip())
print(len(字頻))
字頻[0:5]
# 第一個是什麽鬼
字頻.pop(0)
字頻[0:5]

# 爬取字符

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [ ]:
# 字頻[799]
# # 確認某個位置是不是某個字
# # double-check

In [ ]:
字頻.index('杵')
# # 這個用來檢查現在爬蟲到第幾個字了
# # 在爬蟲過程中我是在另外一個ipynb裏檢查的

In [ ]:
# for zi in 字頻[79:100]:
for zi in 字頻[5985:6001]:
#     print(zi)
    zi_parse = urllib.parse.quote(zi)
    url = 'https://www.zdic.net/zd/zx/qx/' + zi_parse
    r = requests.get(url,headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
        jw = soup.find(name='div',attrs={"class": "zy1"})
        img_list = jw.find_all(name='img')
        
        headers['Referer'] = url

        for img in img_list[0:1]:
            # 暫時只要第一個
            src = 'https:'+ img.get('data-original')
            res = requests.get(url=src,headers=headers)
            filename = zi+'.svg'
            with open(filename,'wb') as f:
                f.write(res.content)        
    except:
        print(str(字頻.index(zi)) + zi + '沒有秦系文字')
        time.sleep(10)
        continue
    time.sleep(11)

## 查漏補缺

In [ ]:
# GB2312全部單字
GB2312 = []
with open('GB2312.txt','r',encoding='utf8') as gb:
    lines = gb.readlines()
    for line in lines:
        for word in line.strip():
            GB2312.append(word)

In [ ]:
converter = opencc.OpenCC('t2s.json')

In [ ]:
# converter = opencc.OpenCC('t2s.json')
# converter.convert('汉字')  # 漢字
zipin_s = []
for char in 字頻:
    char_s = converter.convert(char)
    zipin_s.append(char_s)
zipin_lack = list(set(GB2312).difference(set(zipin_s)))
# 字頻中所有字符簡化，求差，備用（需要補充爬取，雖然幾乎不會有字）

In [ ]:
print(len(zipin_lack))
print(zipin_lack)

In [ ]:
s2t_converter = opencc.OpenCC('s2t.json')

In [ ]:
# s2t_converter = opencc.OpenCC('s2t.json')

for zi_s in zipin_lack:
    zi = s2t_converter.convert(zi_s)
    # 先转成繁体字，按照繁体字比较靠谱些
    # 反正这些是之前字频里面没有的
    # 早知道就直接用GB2312爬了
    # 最初只是想搞几个常用字而已
    # 没想到后来心变大了
    zi_parse = urllib.parse.quote(zi)
    url = 'https://www.zdic.net/zd/zx/qx/' + zi_parse
    r = requests.get(url,headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html.parser')
    try:
        jw = soup.find(name='div',attrs={"class": "zy1"})
        img_list = jw.find_all(name='img')
        
        headers['Referer'] = url
        # 更新请求头，否则403
        
        for img in img_list[0:1]:
            # 暫時只要第一個
            # 其实以后可以做一个alternative字体
            # 很多英文字体都有防止重复出现的替换字型
            
            src = 'https:'+ img.get('data-original')
            res = requests.get(url=src,headers=headers)
            filename = zi+'.svg'
            with open(filename,'wb') as f:
                f.write(res.content)        
    except:
        print(str(zipin_lack.index(zi_s)) + zi + '沒有秦系') # 手動查索引太累了
        time.sleep(15)
        continue
    time.sleep(8)

# 鏡像給簡體字

In [ ]:
一簡對一繁之簡體字 = []
一簡對多繁之簡體字 = []
with open('STCharacters.txt','r',encoding='utf8') as st:
    lines = st.readlines()
    for line in lines:
        a_dict = line.split('\t')
        k_s = a_dict[0]
        v_t = a_dict[1].split(' ')
        if len(v_t) == 1:
            一簡對一繁之簡體字.append(k_s)
        else:
            一簡對多繁之簡體字.append(k_s)

In [ ]:
s2t_dict = {}
with open('STCharacters.txt','r',encoding='utf8') as st:
    lines = st.readlines()
    for line in lines:
        a_dict = line.split('\t')
        s = a_dict[0]
        t = a_dict[1].strip()
        s2t_dict[s] = t

In [ ]:
L = []
for home, dirs, files in os.walk('qx_watermark_removed/'): 
     for file in files:
        L.append(os.path.join(home, file))
# len(L)
list_of_svg_char = []
for l in L:
    char = l.split('.')[0][-1]
    list_of_svg_char.append(char)
# a = 0
will_be_mirrored_s_char = []
for s in list_of_svg_char:
    ss = converter.convert(s)
    if ss not in 一簡對多繁之簡體字 and ss not in list_of_svg_char:
#     if s2t_dict[s] in list_of_svg_char:
#         a += 1
#         if s not in list_of_svg_char:
        # 要保證這個簡體字型不僅有對應繁體字，
        # 還要保證它本身不在已有的svg之内
        will_be_mirrored_s_char.append(ss)
            # 然後才鏡像之
# print(a)
len(will_be_mirrored_s_char)
# 之後在js裏面鏡像，不複製本地文件
# 在最後輸出之前要先去水印的
# # 具體：
# for s_char in will_be_mirrored_s_char:
#     jscode = "font.setSvg('" + s_char + "', fs.readFileSync('去水印/" + s2t_dict[s] + ".svg').toString())\n"

# 去掉水印

In [ ]:
# 將爬取的全部文件放到“去水印”文件夾中
# https://blog.csdn.net/qq_37174526/article/details/89489212
from xml.dom.minidom import parse

def 去水印(filename):
    svg = parse(filename)
    rootNode = svg.documentElement
#     print(rootNode.nodeName)
    gs = rootNode.getElementsByTagName("g")
    for g in gs:
        if g.hasAttribute('fill'):
            color = g.getAttribute('fill')
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(g)

    path = rootNode.getElementsByTagName("path")
    for p in path:
        if p.hasAttribute('fill'):
            color = p.getAttribute('fill')
    #         print(color[-2:])
            if color[-2:] in ['fa','fb','fc']:
#                 print(color)
                rootNode.removeChild(p)
    # print(rootNode.toxml())
    with open(filename,'w') as f:
        f.write(rootNode.toxml())
        
## 注釋：
# 水印的位置和中英文是隨機的，但顔色是近似的，故而利用fill的顔色來判斷是否是水印
# 在瀏覽器中不可見，但是在轉換爲字體之後，會顯現出顔色（單色）
# 所以利用xml的優勢去除水印

In [ ]:
# 獲取所有文件名
# 懶得寫代碼，沒有確認文件擴展名，
# 所以需要手動刪除測試中生成的其他文件
# L = []
# for home, dirs, files in os.walk('去水印'): 
#      for file in files:
#         L.append(os.path.join(home, file))
# 這一步已經挪到了上面


len(L)
# https://www.cnblogs.com/frisk/p/11567175.html

In [ ]:
for l in L:
    去水印(l)

# 輸出Node.js代碼

In [ ]:
with open('create_qx_font.js','w',encoding='utf8') as f:
    f.write('''
var fontCarrier = require('font-carrier')\n
var font = fontCarrier.create()\n
var fs = require('fs')\n
''')
    # https://stackoverflow.com/questions/29835440/meteor-referenceerror-fs-is-not-defined
    for l in L:
        char = l.split('.')[0][-1]
        jscode = "font.setSvg('" + char + "', fs.readFileSync('" + l.replace('\\','/') + "').toString())\n"
        f.write(jscode)
    for s_char in will_be_mirrored_s_char:
        t_char = s2t_converter.convert(s_char)
        if t_char in list_of_svg_char:
            jscode = "font.setSvg('" + s_char + "', fs.readFileSync('qx_watermark_removed//" + t_char + ".svg').toString())\n"
            f.write(jscode)

    f.write('''
font.output({
    path: './iconfont'
})
    ''')

# 後續操作
打開PowerShell, 安裝font-carrier后
輸入
```
node create_font.js
```
即可獲得輸出的多種格式的字體文件